In [1]:
# Importing libraries
import pandas as pd
import os
from datetime import datetime
import shutil
from pathlib import Path

#### Update cell below if you are a new user

In [2]:
# Where the files will be loaded from 
# change the location below to make sure that the file path is correct for your computer
onedrive_base = Path(os.environ.get('onedrive' , 'C:/Users/azona/OneDrive - Cerity Partners'))
process_base = onedrive_base / 'Portfolio Management Team' / '2 - Monthly AUM and Performance' / 'DEV' / '2.1.1 - DEV Addepar Queries'
output = process_base / '2 - Outputs'

input_directory = onedrive_base / process_base
output_directory = onedrive_base / process_base / output
#input_directory = 'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\\DEV\\2.1.1 - DEV Addepar Queries'
#output_path = 'C:\\Users\\azona\\OneDrive - Cerity Partners\\Portfolio Management Team\\2 - Monthly AUM and Performance\\DEV\\2.1.1 - DEV Addepar Queries\\2 - Outputs'

#### Update cell below if you need to update list of firms to be included in the organization transformation process

In [3]:
# Organizations that should be included
included_organizations_list = ['Holdings Algonquin.xlsx', 'Holdings B&J.xlsx', 'Holdings Bainco.xlsx']

# Rows that have special treatment 
values_no_roll_up = ['Unassigned', 'Roll up - Non SMA']

In [4]:
# Import all Excel files starting with "Holdings", in our include list, and are excel files
excel_files = [f for f in os.listdir(input_directory) 
if 
# line below might be redundant
f.startswith("Holdings") 
and 
# line below might be redundant
f.endswith(('.xlsx', '.xls'))
and 
f in included_organizations_list]

# As a QC check let's see what is there
print('List of files imported: ', excel_files)
print('Number of files in the included list: ', len(included_organizations_list))

List of files imported:  ['Holdings Algonquin.xlsx', 'Holdings B&J.xlsx', 'Holdings Bainco.xlsx']
Number of files in the included list:  3


In [ ]:
# Import all Excel files into dictionary
dataframes = {}
for file in excel_files:
    # Load in excel files
    file_path = os.path.join(input_directory, file)
    key_name = os.path.splitext(file)[0]
    data = pd.read_excel(file_path)
    
    # Create 'organization' column by extracting last word from file names
    last_word = key_name.split(' ', 1)[-1]
    organization_name = f"*{last_word}"
    data['Organization'] = organization_name
    
    # Add "row_count" per for filter
    data['row_count'] = data.groupby('CP SMA ID').cumcount() + 1
    
    # First, this code filters down to the rows that have a "SMA ID" that should not be rolled up and filtering down to the first row, which is the top of the rollup in the Excel doc
    # Then, it goes to the values in the "no roll up" variable and removes the top of the roll up, the first row, and keeps the rest
    data = data[((~data['CP SMA ID'].isin(values_no_roll_up)) & (data['row_count'] == 1 ) | (data['CP SMA ID'].isin(values_no_roll_up)) & (data['row_count'] != 1) ) ]
    

    # Getting rid of the total row at the bottom
    data = data[data['CP SMA ID'] != 'Total']
    # Store data
    dataframes[key_name] = data

# Combine all data into one variable
combined_data = pd.concat(dataframes.values(), ignore_index=True)

In [ ]:
# Establishing the columns to drop
columns_2_drop = ['CP SMA ID', 'row_count']

# Removing columns that are not needed in the final output
data = combined_data.drop(columns = columns_2_drop)

# Sorting the columns in the sequence consistent with the way the output is to be seen
columns_sequence = ['Organization' , 'CP SMA ID', 'Adjusted Value (No Div, USD)', 'Adjusted Net Cash Flow (YTD, No Div, USD)']
data = data[columns_sequence]

In [8]:
# Establishing output path locations
output_file = os.path.join(output_directory, "testing.xlsx")

# Publish extract
data.to_excel(output_file, sheet_name='sheet1', index=False)

### If you want to double check some organizations and what SMA information they have, run the cell below and then check against the script that looks at individual organizations to make sure you have the same count of rows

In [ ]:
# Check against the script that runs the individual excel sheet to make sure you have the same number of records
count = combined_df[combined_df['Organization'] == '*Cleveland']
value_count = count['SMA ID'].value_counts()
value_count.sum()